In [25]:
import pandas as pd

# Load data

In [26]:
df_historical_data = pd.read_csv('fifa_worldcup_matches.csv')
df_fixture = pd.read_csv('fifa_worldcup_fixture.csv')
df_missing_data = pd.read_csv('fifa_worldcup_missing_data.csv')

# 1. Data cleaning

## 1.1 Cleaning df_fixture

In [27]:
df_fixture['home'] = df_fixture['home'].str.strip()
df_fixture['away'] = df_fixture['away'].str.strip()

## 1.2 Cleaning df_missing_data and adding it to df_historical

In [28]:
# df_missing_data[df_missing_data['home'].isnull()]
df_missing_data.dropna(inplace=True)
df_historical_data = pd.concat([df_historical_data, df_missing_data], ignore_index=True)
df_historical_data.drop_duplicates(inplace=True)
df_historical_data.sort_values('year', inplace=True)
df_historical_data

,home,score,away,year
0,France,4–1,Mexico,1930
17,Uruguay,4–2,Argentina,1930
16,Uruguay,6–1,Yugoslavia,1930
15,Argentina,6–1,United States,1930
14,Paraguay,1–0,Belgium,1930
...,...,...,...,...
419,Brazil,2–0,Costa Rica,2018
420,Serbia,1–2,Switzerland,2018
421,Serbia,0–2,Brazil,2018
408,France,1–0,Peru,2018


## 1.3 Cleaning df_historical 

#### --------------------------------------- deleting match with walk over ---------------------------------------------

In [29]:
delete_index = df_historical_data[df_historical_data['home'].str.contains('Sweden') &
                                  df_historical_data['away'].str.contains('Austria')].index

df_historical_data.drop(index=delete_index, inplace=True)


#### -------------------------------------- cleanning score and home/away columns ----------------------------------

In [30]:
df_historical_data['score'] = df_historical_data['score'].str.replace('[^\d–]', '', regex=True)
df_historical_data['home'] = df_historical_data['home'].str.strip() # clean blank spaces: Yugoslavia twice
df_historical_data['away'] = df_historical_data['away'].str.strip()

#### -------------------------------- splitting score columns into home and away goals and dropping score column ---------------------------------

In [31]:
df_historical_data[['HomeGoals', 'AwayGoals']] = df_historical_data['score'].str.split('–', expand=True)
df_historical_data.drop('score', axis=1, inplace=True)

#### ------------------------------------------ renaming columns and changing data types --------------------------------------

In [32]:
df_historical_data.rename(columns={'home': 'HomeTeam', 'away': 'AwayTeam', 
                                   'year':'Year'}, inplace=True)
df_historical_data = df_historical_data.astype({'HomeGoals': int, 'AwayGoals':int, 'Year': int})

#### ------------------------------------------ creating new column "totalgoals" ------------------------------------------------- 

In [33]:
df_historical_data['TotalGoals'] = df_historical_data['HomeGoals'] + df_historical_data['AwayGoals']
df_historical_data

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,Mexico,1930,4,1,5
17,Uruguay,Argentina,1930,4,2,6
16,Uruguay,Yugoslavia,1930,6,1,7
15,Argentina,United States,1930,6,1,7
14,Paraguay,Belgium,1930,1,0,1
...,...,...,...,...,...,...
419,Brazil,Costa Rica,2018,2,0,2
420,Serbia,Switzerland,2018,1,2,3
421,Serbia,Brazil,2018,0,2,2
408,France,Peru,2018,1,0,1


# 2. Exporting clean dataframes

In [34]:
df_historical_data.to_csv('clean_fifa_worldcup_matches.csv',index=False)
df_fixture.to_csv('clean_fifa_worldcup_fixture.csv',index=False)

# 3. Verifications

## 3.1 verify number of matches per competition

In [38]:
years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974,
         1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014,
         2018]

for year in years:
    print(year, len(df_historical_data[df_historical_data['Year']==year]))

1930 18
1934 17
1938 18
1950 22
1954 26
1958 35
1962 32
1966 32
1970 32
1974 38
1978 38
1982 52
1986 52
1990 52
1994 52
1998 64
2002 64
2006 64
2010 64
2014 64
2018 64


## 3.2 verify data collected for a team 

In [41]:
print(df_historical_data[df_historical_data['HomeTeam'].str.contains('Morocco')])
print(df_historical_data[df_historical_data['AwayTeam'].str.contains('Morocco')])

    HomeTeam      AwayTeam  Year  HomeGoals  AwayGoals  TotalGoals
222  Morocco  West Germany  1986          0          1           1
631  Morocco        Poland  1986          0          0           0
708  Morocco   Netherlands  1994          1          2           3
710  Morocco        Norway  1998          2          2           4
399  Morocco          Iran  2018          0          1           1
         HomeTeam AwayTeam  Year  HomeGoals  AwayGoals  TotalGoals
477          Peru  Morocco  1970          3          0           3
476  West Germany  Morocco  1970          2          1           3
480      Bulgaria  Morocco  1970          1          1           2
636      Portugal  Morocco  1986          1          3           4
633       England  Morocco  1986          0          0           0
703       Belgium  Morocco  1994          1          0           1
705  Saudi Arabia  Morocco  1994          2          1           3
712        Brazil  Morocco  1998          3          0        